In [ ]:
# 一次爬取多个城市的天气信息，有bug导致这个文件读出的.csv有空白

import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import json
from citydic import *
from citylist import *

i = 0
citys = []
# taiwan = {'高雄': '59554', '屏东': '71301', '花莲': '71305', '嘉义': '71298', '苗栗': '71302', '南投': '71304', '台北': '71294', '桃园': '71294', '新竹': '71294', '宜兰': '71294', '台南': '71299', '台东': '71300', '云林': '71306', '彰化': '71303', '台中': '71082'}
# 遍历得到城市的各个时间的数据
for key,value in citydic.items():
    for year in range(2020,2024):
        for month in range(1,13):
            try:
                url = 'https://tianqi.2345.com/Pc/GetHistory?areaInfo[areaId]='+str(value)+'&areaInfo[areaType]=2&date[year]='+str(year)+'&date[month]='+str(month)
                
                # 爬取并解包
                response = requests.get(url)
                html_content = response.text
                data = json.loads(html_content)
                html_content = data['data']
            
                # 通过bs4对网页源代码进行读取和解析
                soup = BeautifulSoup(html_content, 'html.parser')
                table = soup.find('table')
                rows = table.find_all('tr')
                data = []
                for row in rows:
                    cols = row.find_all('td')
                    cols = [ele.text.strip() for ele in cols]
                    data.append([ele for ele in cols if ele])

                # 写入.csv文件
                with open('tianqi_qu1/'+ str(key) +'_empty.csv', 'a') as f:
                    writer = csv.writer(f)
                    writer.writerows(data)
            
            # 抛去异常
            except:
                pass
    
    # 计数
    print(i)
    i = i + 1
    citys.append(str(key))
        
with open("finishCity.py", "w") as f:
    f.write("finishCity = " + str(citys))
    
print('finished!')

In [ ]:
# 批量去除.csv文件中的空白行

import os
import csv


# 多个地区取空白行并重命名
directory = "tianqi_qu"
directory2 = ""

for filename in os.listdir(directory):
    if filename.endswith(".csv") and "_empty" in filename:
        file_path = os.path.join(directory, filename)
        with open(file_path, "r") as csvfile:
            reader = csv.reader(csvfile)
            lines = [line for line in reader if any(line)]
        new_filename = filename.replace("_empty", "")
        new_file_path = os.path.join(directory2, new_filename)
        with open(new_file_path, "w", newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerows(lines)



In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import json
from citydic import *

# 爬取单个城市的天气（直辖市和港澳台）
key = "丹凤"

for year in range(2020,2024):
    for month in range(1,13):
        try:
            url = 'https://tianqi.2345.com/Pc/GetHistory?areaInfo[areaId]='+str(citydic[key])+'&areaInfo[areaType]=2&date[year]='+str(year)+'&date[month]='+str(month)

            response = requests.get(url)
            html_content = response.text
            data = json.loads(html_content)
            html_content = data['data']


            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find('table')
            rows = table.find_all('tr')
            data = []
            for row in rows:
                cols = row.find_all('td')
                cols = [ele.text.strip() for ele in cols]
                data.append([ele for ele in cols if ele])


            with open('tianqi_qu/'+ key +'_empty.csv', 'a') as f:
                writer = csv.writer(f)
                writer.writerows(data)

        except:
            pass

print(1)

In [ ]:
# 获取城市id的代码

import requests
import re

# url里读取到的是所有县级行政单位的id，共3380个
url = 'http://tianqi.2345.com/tqpcimg/tianqiimg/theme4/js/citySelectData2.js'

response = requests.get(url)

text = response.text

# 下面的text里的是除直辖市与港澳台以外的地级市的id
# text = "prov[10] = '58321-H 合肥-58321|58424-A 安庆-58424|58221-B 蚌埠-58221|58102-B 亳州-58102|58427-C 池州-58427|58236-C 滁州-58236|58203-F 阜阳-58203|58116-H 淮北-58116|70931-H 黄山-70931|58224-H 淮南-58224|58311-L 六安-58311|58336-M 马鞍山-58336|58122-S 宿州-58122|58429-T 铜陵-58429|58334-W 芜湖-58334|58433-X 宣城-58433'prov[13] = '58847-F 福州-58847|71415-D 钓鱼岛-71415|58927-L 龙岩-58927|58834-N 南平-58834|58846-N 宁德-58846|58946-P 莆田-58946|59131-Q 泉州-59131|58828-S 三明-58828|59134-X 厦门-59134|59126-Z 漳州-59126'prov[14] = '52889-L 兰州-52889|56080-G 甘南-56080|60472-L 陇南-60472|52896-B 白银-52896|52995-D 定西-52995|52533-J 酒泉-52533|52675-J 金昌-52675|71129-J 嘉峪关-71129|52984-L 临夏-52984|53915-P 平凉-53915|53923-Q 庆阳-53923|57006-T 天水-57006|52679-W 武威-52679|52652-Z 张掖-52652'prov[15] = '59287-G 广州-59287|59312-C 潮州-59312|59289-D 东莞-59289|59288-F 佛山-59288|59293-H 河源-59293|59297-H 惠州-59297|59473-J 江门-59473|59315-J 揭阳-59315|59659-M 茂名-59659|59109-M 梅州-59109|59280-Q 清远-59280|59493-S 深圳-59493|59501-S 汕尾-59501|59316-S 汕头-59316|59082-S 韶关-59082|59663-Y 阳江-59663|59471-Y 云浮-59471|59658-Z 湛江-59658|59485-Z 中山-59485|59488-Z 珠海-59488|59278-Z 肇庆-59278'prov[16] = '59431-N 南宁-59431|59211-B 百色-59211|59644-B 北海-59644|59425-C 崇左-59425|59635-F 防城港-59635|57957-G 桂林-57957|59249-G 贵港-59249|59023-H 河池-59023|59065-H 贺州-59065|59242-L 来宾-59242|59046-L 柳州-59046|59632-Q 钦州-59632|59265-W 梧州-59265|59453-Y 玉林-59453'prov[17] = '57816-G 贵阳-57816|57825-Q 黔东南-57825|57827-Q 黔南-57827|57806-A 安顺-57806|57707-B 毕节-57707|56693-L 六盘水-56693|70148-Q 黔西南-70148|57741-T 铜仁-57741|57713-Z 遵义-57713'prov[18] = '59758-H 海口-59758|59945-B 保亭-59945|59848-B 白沙-59848|59843-C 澄迈-59843|59847-C 昌江-59847|59838-D 东方-59838|59845-D 儋州-59845|59851-D 定安-59851|59940-L 乐东-59940|59954-L 陵水-59954|59842-L 临高-59842|59849-Q 琼中-59849|59855-Q 琼海-59855|71444-S 三沙-71444|59948-S 三亚-59948|59854-T 屯昌-59854|59951-W 万宁-59951|59856-W 文昌-59856|60651-W 五指山-60651'prov[19] = '53698-S 石家庄-53698|54602-B 保定-54602|54616-C 沧州-54616|54423-C 承德-54423|53892-H 邯郸-53892|54702-H 衡水-54702|54515-L 廊坊-54515|54449-Q 秦皇岛-54449|54534-T 唐山-54534|53798-X 邢台-53798|54401-Z 张家口-54401'prov[20] = '57083-Z 郑州-57083|53898-A 安阳-53898|53990-H 鹤壁-53990|53978-J 济源-53978|53982-J 焦作-53982|57091-K 开封-57091|57186-L 漯河-57186|57073-L 洛阳-57073|57178-N 南阳-57178|54900-P 濮阳-54900|71128-P 平顶山-71128|58005-S 商丘-58005|57051-S 三门峡-57051|57089-X 许昌-57089|57297-X 信阳-57297|53986-X 新乡-53986|57195-Z 周口-57195|57290-Z 驻马店-57290'prov[21] = '50953-H 哈尔滨-50953|50442-D 大兴安岭-50442|50842-D 大庆-50842|50468-H 黑河-50468|50775-H 鹤岗-50775|50978-J 鸡西-50978|50873-J 佳木斯-50873|54094-M 牡丹江-54094|50973-Q 七台河-50973|50745-Q 齐齐哈尔-50745|50853-S 绥化-50853|50884-S 双鸭山-50884|50774-Y 伊春-50774'prov[22] = '57494-W 武汉-57494|57447-E 恩施-57447|57496-E 鄂州-57496|58407-H 黄石-58407|57498-H 黄冈-57498|57377-J 荆门-57377|57476-J 荆州-57476|57475-Q 潜江-57475|57362-S 神农架-57362|57381-S 随州-57381|57252-S 十堰-57252|57483-T 天门-57483|57482-X 孝感-57482|57485-X 仙桃-57485|57278-X 襄阳-57278|57590-X 咸宁-57590|57461-Y 宜昌-57461'prov[23] = '57687-C 长沙-57687|60011-X 湘西-60011|57662-C 常德-57662|57972-C 郴州-57972|57872-H 衡阳-57872|57749-H 怀化-57749|57763-L 娄底-57763|70356-Q 黔阳-70356|57766-S 邵阳-57766|57773-X 湘潭-57773|57865-Y 永州-57865|57674-Y 益阳-57674|57584-Y 岳阳-57584|57780-Z 株洲-57780|57558-Z 张家界-57558'prov[24] = '54161-C 长春-54161|54371-B 白山-54371|50936-B 白城-50936|54172-J 吉林-54172|54260-L 辽源-54260|50949-S 松原-50949|54157-S 四平-54157|54363-T 通化-54363|71532-Y 延边-71532'prov[25] = '58238-N 南京-58238|58343-C 常州-58343|58141-H 淮安-58141|58044-L 连云港-58044|58259-N 南通-58259|58131-S 宿迁-58131|58357-S 苏州-58357|58246-T 泰州-58246|58354-W 无锡-58354|58027-X 徐州-58027|58151-Y 盐城-58151|58245-Y 扬州-58245|58248-Z 镇江-58248'prov[26] = '58606-N 南昌-58606|58617-F 抚州-58617|57993-G 赣州-57993|58502-J 九江-58502|58527-J 景德镇-58527|57799-J 吉安-57799|57786-P 萍乡-57786|58637-S 上饶-58637|57796-X 新余-57796|58627-Y 鹰潭-58627|57793-Y 宜春-57793'prov[27] = '54342-S 沈阳-54342|54339-A 鞍山-54339|54346-B 本溪-54346|54433-C 朝阳-54433|54662-D 大连-54662|54497-D 丹东-54497|54353-F 抚顺-54353|54237-F 阜新-54237|54453-H 葫芦岛-54453|54337-J 锦州-54337|54347-L 辽阳-54347|54338-P 盘锦-54338|54249-T 铁岭-54249|54471-Y 营口-54471'prov[28] = '53463-H 呼和浩特-53463|60150-W 乌兰察布-60150|60149-X 锡林郭勒-60149|60356-A 阿拉善盟-60356|53446-B 包头-53446|54218-C 赤峰-54218|71109-E 鄂尔多斯-71109|71108-H 呼伦贝尔-71108|60002-L 巴彦淖尔-60002|54135-T 通辽-54135|53512-W 乌海-53512|60001-X 兴安盟-60001'prov[29] = '53614-Y 银川-53614|53817-G 固原-53817|53518-S 石嘴山-53518|53612-W 吴忠-53612|53704-Z 中卫-53704'prov[30] = '52866-X 西宁-52866|71113-H 海西-71113|71729-G 果洛-71729|71727-H 海东-71727|71728-H 海南-71728|71112-H 海北-71112|71114-H 黄南-71114|70552-Y 玉树-70552'prov[31] = '54823-J 济南-54823|54734-B 滨州-54734|54714-D 德州-54714|54736-D 东营-54736|54906-H 菏泽-54906|54915-J 济宁-54915|54806-L 聊城-54806|54938-L 临沂-54938|54857-Q 青岛-54857|54945-R 日照-54945|54827-T 泰安-54827|54843-W 潍坊-54843|54774-W 威海-54774|54765-Y 烟台-54765|54830-Z 淄博-54830|58024-Z 枣庄-58024'prov[32] = '53772-T 太原-53772|53882-C 长治-53882|53487-D 大同-53487|71115-J 晋中-71115|53976-J 晋城-53976|53868-L 临汾-53868|71037-L 吕梁-71037|53578-S 朔州-53578|53674-X 忻州-53674|53782-Y 阳泉-53782|53959-Y 运城-53959'prov[33] = '57036-X 西安-57036|57245-A 安康-57245|57016-B 宝鸡-57016|57127-H 汉中-57127|71031-S 商洛-71031|53947-T 铜川-53947|57045-W 渭南-57045|57048-X 咸阳-57048|71199-Y 杨凌-71199|53845-Y 延安-53845|53646-Y 榆林-53646'prov[35] = '56294-C 成都-56294|56171-A 阿坝-56171|57313-B 巴中-57313|56198-D 德阳-56198|57328-D 达州-57328|57415-G 广安-57415|56146-G 甘孜-56146|57206-G 广元-57206|57602-L 泸州-57602|56386-L 乐山-56386|71118-L 凉山-71118|56391-M 眉山-56391|56196-M 绵阳-56196|57411-N 南充-57411|57504-N 内江-57504|56666-P 攀枝花-56666|57405-S 遂宁-57405|56492-Y 宜宾-56492|56287-Y 雅安-56287|56396-Z 自贡-56396|56298-Z 资阳-56298'prov[38] = '55591-L 拉萨-55591|55437-A 阿里-55437|56137-C 昌都-56137|56312-L 林芝-56312|70774-N 那曲-70774|55578-R 日喀则-55578|55597-S 山南-55597'prov[40] = '51463-W 乌鲁木齐-51463|51656-B 巴音郭楞-51656|51704-K 克州-51704|51730-A 阿拉尔-51730|51076-A 阿勒泰-51076|51628-A 阿克苏-51628|71708-B 巴州-71708|71710-B 博尔塔拉-71710|71714-B 北屯-71714|51238-B 博州-51238|51368-C 昌吉-51368|51828-H 和田-51828|52203-H 哈密-52203|71717-K 可克达拉-71717|51243-K 克拉玛依-51243|72108-K 昆玉市-72108|51709-K 喀什-51709|71716-S 双河-71716|51356-S 石河子-51356|71712-T 图木舒克-71712|71715-T 铁门关-71715|51573-T 吐鲁番-51573|51133-T 塔城-51133|71713-W 五家渠-71713|71709-Y 伊犁-71709'prov[41] = '56778-K 昆明-56778|70908-D 迪庆-70908|60839-X 西双版纳-60839|56748-B 保山-56748|56768-C 楚雄-56768|56751-D 大理-56751|71126-D 德宏-71126|56975-H 红河-56975|56651-L 丽江-56651|56951-L 临沧-56951|71127-N 怒江-71127|70887-P 普洱-70887|56783-Q 曲靖-56783|56994-W 文山-56994|56875-Y 玉溪-56875|56586-Z 昭通-56586'prov[42] = '58457-H 杭州-58457|58450-H 湖州-58450|58452-J 嘉兴-58452|58549-J 金华-58549|58646-L 丽水-58646|58465-N 宁波-58465|58633-Q 衢州-58633|58453-S 绍兴-58453|58651-T 台州-58651|58659-W 温州-58659|58477-Z 舟山-58477'"

pattern = re.compile(r'(\w+)\s+([\u4e00-\u9fa5]+)-(\w+)')
result = re.findall(pattern, text)

data = {}
for item in result:
    data[item[1]] = item[2]

with open("citydic1.py", "w") as f:
    f.write("citydic = " + str(data))

print(1)